# Setting up script

###### or feeding the spider

In [1]:
#Library section,or....just library, like ya know...a library
import pandas as pd
from parsel import Selector
import time
import random
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

In [2]:
from credentials import credentials

In [3]:
#credentials to sign in
email_id = credentials['email_id']
password_0 = credentials['password']

# Sign-in and scouting  

###### and pretense of a normal

In [4]:
#Start Chrome webdriver
driver = webdriver.Chrome()


#Open LinkedIn website
driver.get('https://www.linkedin.com')

#Locate and enter email/username
username = driver.find_element(By.ID, 'session_key')
username.send_keys(email_id)

#Add a short delay (like two people in a large coat)
sleep(0.5)

#Locate and enter password
password = driver.find_element(By.ID, 'session_password')
password.send_keys(password_0)

#Locate and click the sign-in button
"""
note to self: if there are any errors of not finding the element using different data, chances
are that you will have to get some delay before the click using WebDriverWait
"""

sign_in_button = driver.find_element(By.XPATH, '//*[@type="submit"]')
sign_in_button.click()

#Add a longer delay to allow time for the sign-in process
sleep(6)



# Profile extraction from search bar

In [5]:
#To find the search bar element
search_field = driver.find_element(By.XPATH, '//*[@id="global-nav-typeahead"]/input')
full_name = input("Enter the name you wish to look up: ")
search_field.send_keys(f'{full_name}')

#Search
search_field.send_keys(Keys.RETURN)


Enter the name you wish to look up:  daksh patel


In [6]:
#find people button by xpath
sleep(5)
people = driver.find_element(By.XPATH,'//*[@id="search-reusables__filters-bar"]/ul/li[1]/button')
#button click
people = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="search-reusables__filters-bar"]/ul/li[1]/button')))

people.click()
sleep(2)

In [7]:
#getting URLs of the profiles
profiles = driver.find_elements(By.CLASS_NAME, 'app-aware-link')
all_URL = []
for profile in profiles:
    profile_URL = profile.get_attribute('href')
    if profile_URL not in all_URL:
        all_URL.append(profile_URL)

print('done')

done


In [8]:
main_profile_URL = []
for items in all_URL:
    if len(items)>40:
        main_profile_URL.append(items)
#we now have all the ammunition we need to go on with our process

###### setting the stage for main event

# Information extraction from profile

In [9]:
def validate_field(field):
    if field:
        pass
    else:
        field = "no results"
    return field

In [10]:
#something I will try later on, as for now, this feature is on hold
# def count_certificates(url):
#     
    

#     try:
        
#         try:
            
#             see_more = driver.find_element(By.ID,'navigation-index-see-all-licenses-and-certifications')
#             see_more_button.click()
#         except:
#             pass
            
#         ul_element = driver.find_element(By.XPATH, '//div[@id="yourElementId"]/span[@aria-hidden="true"]')
        
        

#         
#         li_count = len(ul_element.find_elements(By.TAG_NAME, 'li'))
#         return li_count

#     except:
#         pass


In [11]:
profiledata =[]
for link in main_profile_URL:
    driver.get(link)
    time.sleep(random.uniform(2.5,4.9))
    # some use xpath, some use class names
    """
    note to self: do NOT change any xpath without checking at least 3 times with original webpage elements
    """
    sel = Selector(text=driver.page_source)
    name = sel.xpath('//*[starts-with(@class,"text-heading-xlarge inline t-24 v-align-middle break-words")]/text()').extract_first()
    if name:
        name = name.strip()
    job_title = sel.xpath('//*[starts-with(@class,"text-body-medium break-words")]/text()').extract_first()

    if job_title:
        job_title = job_title.strip()

    
    connections = driver.find_element(By.XPATH,'//p[contains(@class, "wEMnHRWZkPesuVSxhIljYnMUSvZWxU")]/span[@aria-hidden="true"]').text
    if connections:
        connections = connections.strip()

    try:
        company = driver.find_element(By.XPATH,'//*[@id="profile-content"]/div/div[2]/div/div/main/section[1]/div[2]/div[2]/ul/li/button/span').text
    except:
        company = 'None'
    if company:
        company = company.strip()
    location = sel.xpath('//*[starts-with(@class, "text-body-small inline t-black--light break-words")]/text()').extract_first()
    if location:
        location.strip()
    try:
       
        about = driver.find_element(By.ID,"about").find_element(By.XPATH,"..").find_element(By.CLASS_NAME,"display-flex").text   
    except:
        about = 'None'
    if about:
        about = about.strip()
        
    
             
    #to catch any issues, we run all of the fields to our validate_field function
    name = validate_field(name)
    job_title = validate_field(job_title)
    company = validate_field(company)
    location = validate_field(location)
    about = validate_field(about)
    #certificates = validate_field(certificates)
   

    print('\n')
    print('Name: ' + name)
    print('Job Title: ' + job_title)
    print('Company: ' + company)
    print('Location: ' + location)
    print('About: ' + about)
    print('Connections: ' + connections)
    print('URL: ' + link)
    #print('Certificates: ' + certificates)
    print('\n')

    data = {
        'URL': link,
        'Name': name,
        'Job Title': job_title,
        'Company': company,
        'Location':location,
        'About': about,
        'Connections': connections,
        #'Certificates': certificates
        
    }
        
    profiledata.append(data)
    
profiledataframe = pd.DataFrame(profiledata)




Name: Daksh Patel
Job Title: Business Development Manager
Company: Logix Built Solutions Pvt Ltd
Location: 
          Surat, Gujarat, India
        
About: Hello, I'm Daksh Patel, a results-oriented Business Development Manager in the IT/Software Service industry. With expertise in driving client acquisition and market analysis, I consistently surpass targets. My strength lies in cultivating robust client relationships and thriving in collaborative tech environments. Let's connect for innovative solutions and business growth
Hello, I'm Daksh Patel, a results-oriented Business Development Manager in the IT/Software Service industry. With expertise in driving client acquisition and market analysis, I consistently surpass targets. My strength lies in cultivating robust client relationships and thriving in collaborative tech environments. Let's connect for innovative solutions and business growth
Connections: 11 followers
URL: https://www.linkedin.com/in/daksh-patel-7b77322ab?miniProfile

# Final wrap up

In [27]:
for i in range(len(profiledataframe['About'])):
    if "…see more" in profiledataframe['About'][i]:
        profiledataframe['About'][i] = profiledataframe['About'][i].replace('…see more', '')

        

In [30]:
profiledataframe['About'][2]

'Passionate about leveraging data to drive insights and create meaningful solutions. Experienced in diverse roles ranging from content development to sales team management, software development, and business analysis. Skilled in applying statistical techniques and machine learning algorithms to extract valuable insights from complex datasets. Excited about the exponential growth of the data field and the transformative potential of AI.\n\nSkills : Predictive Analytics, Statistical Data Analysis, Natural Language Processing (NLP), Deep learning, Computer Vision, Data analytics, Data visualization, Machine learning, , Cloud computing, Statistics and probability, Data wrangling, Advanced Mathematics. data modeling\n\nLanguages: Python, SQL, C++\n\nLibraries: Pandas, NumPy, Scikit-learn, Tensorflow, Matplotlib, Ggplot2, Seaborn\n\nDatabases: Google BigQuery\n\nTools: JupyterLab, PyCharm, Google Analytics, Microsoft Clarity, Tableau\n'

In [13]:
#create a csv file with all data entries for further usage(like for visualization)
profiledataframe.to_csv('profiledata.csv')

profiledataframe

,URL,Name,Job Title,Company,Location,About,Connections
0,https://www.linkedin.com/in/daksh-patel-7b7732...,Daksh Patel,Business Development Manager,Logix Built Solutions Pvt Ltd,"\n Surat, Gujarat, India\n","Hello, I'm Daksh Patel, a results-oriented Bus...",11 followers
1,https://www.linkedin.com/in/daksh-patel-501702...,Daksh Patel,Student at The Maharaja Sayajirao University o...,The Maharaja Sayajirao University of Baroda,"\n Gujarat, India\n",None,125 followers
2,https://www.linkedin.com/in/daksh-patel-922ba5...,Daksh Patel,Data Scientist | GDSC Tech Lead,Google Developer Student Clubs,"\n Gandhinagar, Gujarat, India\n",Passionate about leveraging data to drive insi...,304 followers
3,https://www.linkedin.com/in/daksh-patel-?miniP...,Daksh Patel,Digital Product Designer,SandCup Design Studio,"\n Ahmedabad, Gujarat, India\n",Creating mindful and intuitive human-centered ...,"1,817 followers"
4,https://www.linkedin.com/in/daksh-patel-aa8699...,Daksh Patel,"""Passionate Innovator | Driving Positive Chang...",Roma Industries,"\n Ahmedabad, Gujarat, India\n",Experienced Director with a demonstrated histo...,83 followers
5,https://www.linkedin.com/in/daksh-patel-5067a1...,Daksh Patel,--,Pandit Deendayal Energy University,"\n Gandhinagar, Gujarat, India\n",I am currently pursuing b.tech in computer eng...,537 followers
6,https://www.linkedin.com/in/daksh-patel-30581b...,Daksh Patel,JAVA DEVELOPER || CORE JAVA || Well Known ...,"LDRP Institute of Technology & Research, Gujar...","\n Gandhinagar, Gujarat, India\n",None,175 followers
7,https://www.linkedin.com/in/daksh-p-0361b3121?...,Daksh P.,no results,None,"\n Gujarat, India\n",An engineer who has a vision to merge data sci...,499 followers
8,https://www.linkedin.com/in/daksh-patel-920785...,Daksh Patel,Associate software Engineer at Thomson Reuters,Thomson Reuters,"\n Ahmedabad, Gujarat, India\n",None,936 followers
9,https://www.linkedin.com/in/-daksh-patel?miniP...,Daksh Patel,SDE @Jio ( DSA | JAVA | LLD | REACTJS | NODEJS...,Jio,"\n Ahmedabad, Gujarat, India\n",-Experienced Software Developer with a demonst...,"2,279 followers"
